In [27]:
import torch
from PIL import Image
import pandas as pd
import numpy as np



In [28]:
from importlib.resources import path

from torch import FloatTensor, LongTensor
from torchvision import transforms

class MNISTDatasetASL(torch.utils.data.Dataset):
    def __init__(self, path):
        
        self.X, self.Y = self.get_data_mapping(path)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(28, scale=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(p = 0.5),
            transforms.RandomInvert(p=0.25),
            transforms.ToTensor(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)]
        )

    def get_data_mapping(self, path):
        dataset = pd.read_csv(path)
        Y = torch.tensor(dataset['label'].values.astype(np.int8))
        X = torch.tensor(dataset.drop('label', axis = 1).values.astype(np.int8)) 
        return (X, Y)    

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        label = self.Y[idx].type(LongTensor)
        input = self.transform(self.X[idx].reshape(1, 28, 28).type(FloatTensor))
        return (input, label)


In [29]:
from sklearn.utils import shuffle


train_path = "data/sign_mnist_train.csv"
test_path = "data/sign_mnist_test.csv"

trainLoader = torch.utils.data.DataLoader(MNISTDatasetASL(train_path), batch_size=64, shuffle = True)
testLoader = torch.utils.data.DataLoader(MNISTDatasetASL(train_path), batch_size=64, shuffle = True)


In [30]:
transform = transforms.Compose([transforms.ToPILImage()])
for img, y in testLoader:
    img = transform(img[0].reshape(1,28,28))

img

## Define Network Architectyre

In [31]:
import torch.nn as nn

class ASLClassifier(nn.Module):
    class ConvBlock(nn.Module):
        def __init__(self, input_c, output_c, kernel_size):
            super().__init__()
            self.block = nn.Sequential(
                nn.Conv2d(input_c, output_c, kernel_size),
                nn.ReLU(),
                nn.MaxPool2d(2,2)
            )
        def forward(self, x):
            return self.block(x)

        
    def __init__(self):
        print("HERE")
        super().__init__()

        self.conv = torch.nn.Sequential(
            self.ConvBlock(1, 6, 3),
            self.ConvBlock(6, 16, 3)
        )
        self.mlp = torch.nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 48),
            nn.ReLU(),
            nn.Linear(48, 25)
        )

    def forward(self, x):
        flat_out = self.conv(x).flatten(start_dim = 1)
        return self.mlp(flat_out)

    def predict(self, x):
        return torch.argmax(self.forward(x), dim=1)

net = ASLClassifier()


HERE


In [32]:
%reload_ext tensorboard
%tensorboard --logdir docs

Reusing TensorBoard on port 6006 (pid 33653), started 12:32:55 ago. (Use '!kill 33653' to kill it.)

## Set up a Training Loop

In [33]:
import torch.utils.tensorboard as tb
from os import path

def train(model, train_loader, test_loader, lr = 0.001, epochs = 2, log_dir = 'docs'):
    
    optimizer = torch.optim.AdamW(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    global_step = 0

    train_logger = tb.SummaryWriter(path.join(log_dir, 'train'))
    valid_logger = tb.SummaryWriter(path.join(log_dir, 'valid'))

    for epoch in range(epochs):
        total_loss = 0
        for X, Y in train_loader:
            X = X.type(torch.FloatTensor)
            output = model(X)
            loss = loss_fn(output, Y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_logger.add_scalar('loss', loss, global_step=global_step)

            global_step += 1

            total_loss += loss
        
        print(f"LOSS AT EPOCH {epoch} : {total_loss}")

        # total_correct = 0
        # count = 0
        # for X, Y in test_loader:
        #     predictions = model.predict(X)

        #     total_correct += (predictions == Y).sum()
        #     count += len(Y)
        # print(f"Valid Accuracy AT EPOCH {epoch} : {total_correct/ count}")
        # valid_logger.add_scalar('accuracy', total_correct/count, global_step=global_step)

net = ASLClassifier()
train(net, trainLoader, testLoader, lr = 0.001, epochs=15)





HERE
LOSS AT EPOCH 0 : 2471.107666015625
Valid Accuracy AT EPOCH 0 : 0.2797304689884186
LOSS AT EPOCH 1 : 1568.4107666015625
Valid Accuracy AT EPOCH 1 : 0.5014023184776306
LOSS AT EPOCH 2 : 1128.689453125
Valid Accuracy AT EPOCH 2 : 0.6232016086578369
LOSS AT EPOCH 3 : 897.0569458007812
Valid Accuracy AT EPOCH 3 : 0.6832271218299866
LOSS AT EPOCH 4 : 769.937744140625
Valid Accuracy AT EPOCH 4 : 0.7376798391342163
LOSS AT EPOCH 5 : 652.2020263671875
Valid Accuracy AT EPOCH 5 : 0.7583682537078857
LOSS AT EPOCH 6 : 587.7547607421875
Valid Accuracy AT EPOCH 6 : 0.7826260924339294
LOSS AT EPOCH 7 : 533.3922119140625
Valid Accuracy AT EPOCH 7 : 0.807721734046936
LOSS AT EPOCH 8 : 475.6503601074219
Valid Accuracy AT EPOCH 8 : 0.8246585130691528
LOSS AT EPOCH 9 : 446.1098937988281
Valid Accuracy AT EPOCH 9 : 0.8271717429161072
LOSS AT EPOCH 10 : 414.5137634277344
Valid Accuracy AT EPOCH 10 : 0.8504461646080017
LOSS AT EPOCH 11 : 396.98095703125
Valid Accuracy AT EPOCH 11 : 0.8621016144752502
L

In [34]:
torch.save(net.state_dict(), "checkpoint.pth")

## Export Code To Onnpix

In [39]:
import onnx
import onnxruntime as ort

testLoader = torch.utils.data.DataLoader(MNISTDatasetASL(test_path), batch_size=1)

net = ASLClassifier()
model = torch.load("checkpoint.pth")
net.load_state_dict(model)

acc_count = 0
count = 0
for X, Y in testLoader:
    predictions = net.predict(X)
    # print(type(predictions == Y))
    acc_count += (predictions == Y).type(torch.LongTensor)
    count += 1

print(f'Torch Model Accuracy (baseline):  {acc_count / count}')


# export to onnx
fname = "asl_model.onnx"
dummy = torch.randn(1, 1, 28, 28)
net(dummy)
torch.onnx.export(net, dummy, fname, input_names=['input'])

# check exported model
model = onnx.load(fname)
onnx.checker.check_model(model)  # check model is well-formed

# create runnable session with exported model
ort_session = ort.InferenceSession(fname)
net = lambda inp: ort_session.run(None, {'input': inp.data.numpy()})[0]


acc_count = 0
count = 0
for X, Y in testLoader:
    # print(net(X))

    predictions = net(X).argmax()
    acc_count += int(predictions == Y)
    count += 1

print(f'Torch Model Accuracy (baseline):  {acc_count / count}')



HERE
Torch Model Accuracy (baseline):  tensor([0.8088])
Torch Model Accuracy (baseline):  0.8064696040156163
